In [1]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import cosine_distances
import numpy as np

In [2]:

import gensim, pickle
from gensim.models.keyedvectors import KeyedVectors
LdaModel = gensim.models.ldamodel.LdaModel


In [3]:
ruta_word_embedding = '../data/wiki.multi.en.vec'
##wordembedding = gensim.models.Word2Vec.load(ruta_word_embedding)
wordembedding = KeyedVectors.load_word2vec_format(ruta_word_embedding, binary=False)

In [8]:
lda_model_collecion_1.print_topics()

[(0,
  '0.081*"trump" + 0.043*"election" + 0.034*"campaign" + 0.031*"leave" + 0.027*"conservative" + 0.024*"vote" + 0.018*"support" + 0.017*"action" + 0.016*"group" + 0.016*"speech"'),
 (1,
  '0.024*"start" + 0.023*"talk" + 0.023*"big" + 0.021*"run" + 0.020*"man" + 0.019*"love" + 0.017*"happen" + 0.015*"guy" + 0.013*"show" + 0.012*"wait"'),
 (2,
  '0.050*"scandal" + 0.044*"company" + 0.035*"data" + 0.022*"security" + 0.021*"report" + 0.019*"firm" + 0.019*"breach" + 0.015*"leak" + 0.013*"affect" + 0.012*"lead"'),
 (3,
  '0.205*"datum" + 0.093*"user" + 0.060*"share" + 0.046*"information" + 0.034*"personal" + 0.027*"message" + 0.026*"collect" + 0.024*"private" + 0.023*"access" + 0.022*"send"'),
 (4,
  '0.089*"datum" + 0.065*"give" + 0.047*"sell" + 0.046*"pay" + 0.041*"make" + 0.034*"money" + 0.033*"business" + 0.033*"people" + 0.030*"free" + 0.022*"company"'),
 (5,
  '0.078*"time" + 0.051*"year" + 0.032*"back" + 0.022*"day" + 0.021*"long" + 0.020*"life" + 0.019*"real" + 0.017*"ago" + 0.01

In [4]:
##Load Gensim Model

lda_model_collecion_1 = LdaModel.load("../data/cambridge_analytica/collection_I/collection_1_gensim.model")
lda_model_collecion_2 = LdaModel.load("../data/cambridge_analytica/collection_I/collection_1_gensim.model")

In [5]:
with open('../data/cambridge_analytica/collection_I/collection_1_sent_topics_sorteddf_mallet_ldamodel', 'rb') as f:
    most_relevant_documents_collection_1 = pickle.load(f)
most_relevant_documents_collection_1 = most_relevant_documents_collection_1[['Topic_Num','Topic_Perc_Contrib','text']]


with open('../data/cambridge_analytica/collection_I/collection_1_sent_topics_sorteddf_mallet_ldamodel', 'rb') as f:
    most_relevant_documents_collection_2 = pickle.load(f)
most_relevant_documents_collection_2 = most_relevant_documents_collection_2[['Topic_Num','Topic_Perc_Contrib','text']]


In [6]:
'''My own tokenizer '''
import unidecode
from string import punctuation
from string import digits
punctuation+="¡¿<>'`"
punctuation+='"'

#Remove digits and puntuaction
remove_digits = str.maketrans(digits, ' '*len(digits))#remove_digits = str.maketrans('', '', digits)
remove_punctuation = str.maketrans(punctuation, ' '*len(punctuation))#remove_punctuation = str.maketrans('', '', punctuation)
remove_hashtags_caracter = str.maketrans('#', ' '*len('#'))
#las palabras de los hashtag se mantiene, pero no el simbolo. 


def text_cleaner(tweet):
    tweet = tweet.translate(remove_digits)
    #tweet = tweet.lower() it wasn't a good idea,, we lost a lot of
    tweet = tweet.translate(remove_punctuation)
    tweet = tweet.translate(remove_hashtags_caracter)
    tweet = tweet.lower()
    tweet = unidecode.unidecode(tweet)
    #tweet = tweet.strip().split()
    #filtered_words = [word for word in tweet if word not in stopWords]
    #corpus[id_tweet]= filtered_words
    #id_tweet+=1
    tweet = tweet.split()
    return tweet


In [7]:
def get_dicts_relevant_keywords_documents(lda_model,df_relevant_documents, n_terms):
    num_topics = lda_model.num_topics
    #create dictionary of top keywords 
    topKeywordsDict = {}
    for topic_id in range(num_topics):
        topKeywordsDict[topic_id] = []
        for term, probability in lda_model.show_topic(topic_id,topn=n_terms):
            topKeywordsDict[topic_id].append({
                "term":term,
                "probability":probability
            })

    #create dictionary of relevant documents
    relevantDocumentsDict = {}
    for index,row in df_relevant_documents.iterrows():
        topic_id = int(row['Topic_Num'])
        if topic_id not in relevantDocumentsDict:
            relevantDocumentsDict[topic_id]=[]
        relevantDocumentsDict[topic_id].append({
            'topic_perc_contrib':row['Topic_Perc_Contrib'],
            'text':row['text']
        })
    return (topKeywordsDict, relevantDocumentsDict)

def getDocumentVector(text, wordembedding):
    #preprocesar    
    #encontrar palabras en word embedding
    #ponderas palabras TF-IDF
    document_vector = 0.0
    words_found = 0.0
    for word in text_cleaner(text):
        if word in wordembedding:
            document_vector+=wordembedding.wv[word] #aqui hay que ponderar
            words_found+=1
    return document_vector/words_found

        
    
def get_topkeywords_relevantdocuments_vectors(wordembedding, lda_model,most_relevant_documents,  n_terms): #n_terms : numero de top keywords a considerar
    topKeywordsDict, relevantDocumentsDict = get_dicts_relevant_keywords_documents(lda_model, most_relevant_documents, n_terms)

    ### Create top keyword vector per topic
    #create keyword vector
    topkeywords_vectors_dict = {}
    num_topics = lda_model.num_topics
    for topic_id in range(num_topics):
        topkeywords_vector = 0
        ranking = 1.0
        for item in topKeywordsDict[topic_id]:
            if item['term'] in wordembedding: #no todas las palabras aparecerán en el ranking, que hacer con el resto
                #print(item['term'], item['probability'])
                topkeywords_vector += wordembedding.wv[item['term']]/ranking
            else:
                print(item['term']," position:",ranking)
            ranking+=1
        topkeywords_vectors_dict[topic_id] = topkeywords_vector
        
    relevantdocuments_vectors_dict = {}
    for topic_id in range(num_topics):
        relevantDocumentsvector = 0.0
        for item in relevantDocumentsDict[topic_id]:
            #quizas esto hacerlo para los primero 100 docs, 500 docs, el resto es un % pequeño que se pierde, optimizar
            #revisar si multiplicar por la contribucion es lo correcto, eso no da 1 o si? OHHHHHH, habria que sacar todooos los docs no solo los 100 primeros
            relevantDocumentsvector+= float(item['topic_perc_contrib'])*getDocumentVector(item['text'], wordembedding)
        relevantdocuments_vectors_dict[topic_id] = relevantDocumentsvector
        
    return (topkeywords_vectors_dict, relevantdocuments_vectors_dict)


In [8]:
def get_topic_vectors(wordembedding, lda_model,most_relevant_documents,  n_terms, lambda_):
    num_topics = lda_model.num_topics
    topkeywords_vectors_dict, relevantdocuments_vectors_dict = get_topkeywords_relevantdocuments_vectors(wordembedding, lda_model,most_relevant_documents,  n_terms)
    final_topic_vectors_dict = dict()
    for topic_id in range(num_topics):
        final_topic_vector = lambda_*topkeywords_vectors_dict[topic_id]+(1-lambda_)*relevantdocuments_vectors_dict[topic_id]
        final_topic_vectors_dict[topic_id] = final_topic_vector
    return final_topic_vectors_dict

In [9]:
def get_matrix(wordembedding, lda_model_1,most_relevant_documents_1,lda_model_2,most_relevant_documents_2, n_terms, lambda_):
    
    final_topic_vectors_dict_1 =  get_topic_vectors(wordembedding, lda_model_1,most_relevant_documents_1,  n_terms, lambda_)
    final_topic_vectors_dict_2 =  get_topic_vectors(wordembedding, lda_model_2,most_relevant_documents_2,  n_terms, lambda_)
    
    topic_similarity_matrix = []
    for i in range(lda_model_1.num_topics):
        row = []
        for j in range(lda_model_2.num_topics):
            topic_i = final_topic_vectors_dict_1[i].reshape(1,-1)
            topic_j = final_topic_vectors_dict_2[j].reshape(1,-1)
            row.append(float(cosine_similarity(topic_i,topic_j)))
        topic_similarity_matrix.append(row)
    topic_similarity_matrix= np.asarray(topic_similarity_matrix)
    return topic_similarity_matrix

In [ ]:
topn = 30
#lambda_ = 0.8


i = 0.0
matrices_dict = dict()
while i <=1.01:
    lambda_ = round(i,2)
    print(lambda_)
    matrix = get_matrix(wordembedding, lda_model_collecion_1, most_relevant_documents_collection_1, lda_model_collecion_2, most_relevant_documents_collection_2,topn, lambda_)
    matrices_dict[lambda_] = matrix
    i+=0.01
 

0.0


C:\Users\gonza\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:52: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
C:\Users\gonza\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:33: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).


deletefacebook  position: 21.0
retweete  position: 14.0
bigdata  position: 1.0
machinelearne  position: 9.0
datascience  position: 13.0
deletefacebook  position: 9.0
twitt  position: 29.0
deletefacebook  position: 21.0
retweete  position: 14.0
bigdata  position: 1.0
machinelearne  position: 9.0
datascience  position: 13.0
deletefacebook  position: 9.0
twitt  position: 29.0
0.01


C:\Users\gonza\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:52: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
C:\Users\gonza\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:33: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).


deletefacebook  position: 21.0
retweete  position: 14.0
bigdata  position: 1.0
machinelearne  position: 9.0
datascience  position: 13.0
deletefacebook  position: 9.0
twitt  position: 29.0
deletefacebook  position: 21.0
retweete  position: 14.0
bigdata  position: 1.0
machinelearne  position: 9.0
datascience  position: 13.0
deletefacebook  position: 9.0
twitt  position: 29.0
0.02


...

deletefacebook  position: 21.0
retweete  position: 14.0
bigdata  position: 1.0
machinelearne  position: 9.0
datascience  position: 13.0
deletefacebook  position: 9.0
twitt  position: 29.0
deletefacebook  position: 21.0
retweete  position: 14.0
bigdata  position: 1.0
machinelearne  position: 9.0
datascience  position: 13.0
deletefacebook  position: 9.0
twitt  position: 29.0
0.48


C:\Users\gonza\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:52: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
C:\Users\gonza\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:33: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).


deletefacebook  position: 21.0
retweete  position: 14.0
bigdata  position: 1.0
machinelearne  position: 9.0
datascience  position: 13.0
deletefacebook  position: 9.0
twitt  position: 29.0
deletefacebook  position: 21.0
retweete  position: 14.0
bigdata  position: 1.0
machinelearne  position: 9.0
datascience  position: 13.0
deletefacebook  position: 9.0
twitt  position: 29.0
0.49


C:\Users\gonza\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:52: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
C:\Users\gonza\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:33: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).


deletefacebook  position: 21.0
retweete  position: 14.0
bigdata  position: 1.0
machinelearne  position: 9.0
datascience  position: 13.0
deletefacebook  position: 9.0
twitt  position: 29.0


In [ ]:
matrices_dict

In [18]:
with open('../data/cambridge_analytica/matrix_collection_1_1', 'wb') as f:
            pickle.dump(matrices_dict, f)

In [11]:
with open('../data/cambridge_analytica/matrix_collection_1_2', 'rb') as f:
    matrix_sankey_2 = pickle.load(f)


In [13]:
matrix_sankey_2[0.8]

array([[0.93399   , 0.77258611, 0.95911622, 0.80941415, 0.82363397,
        0.93731242, 0.87817943, 0.8465876 , 0.78703463, 0.84308201,
        0.68733966, 0.8296324 , 0.83066213, 0.86650491, 0.85298061],
       [0.87455618, 0.81576705, 0.87215126, 0.81436431, 0.81050557,
        0.85118937, 0.89927334, 0.88840425, 0.81510121, 0.92421865,
        0.70527971, 0.92513371, 0.84377515, 0.89781153, 0.82696366],
       [0.83998048, 0.75661427, 0.81549239, 0.88003439, 0.8673358 ,
        0.86852229, 0.84527749, 0.82375681, 0.8605417 , 0.79867864,
        0.7792629 , 0.8093214 , 0.85483277, 0.88069713, 0.9714793 ],
       [0.84105194, 0.80832881, 0.81025606, 0.97125363, 0.86455184,
        0.85414851, 0.87730783, 0.85154277, 0.8416298 , 0.86179429,
        0.77739209, 0.84590256, 0.88172126, 0.87899113, 0.88981771],
       [0.85330963, 0.78228241, 0.8324753 , 0.93865907, 0.8363713 ,
        0.86542916, 0.88884461, 0.85795116, 0.86766207, 0.86610055,
        0.75005126, 0.89091283, 0.87751663, 